In [1]:
import numpy as np

# ステージ２　自然なコードで表現する

## ステップ１１　可変長の引数（順伝播編）
### 11.1 Functionクラスの修正
これまで通り、「１つの変数」を取り、「１つの値」を返すように実装する。<br>
ただし、その引数と戻り値をリストに変更し、そのリストの中に必要な変数を入れるよう修正

In [2]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported')

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [3]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [4]:
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]  # Get data from Variable
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]  # Wrap data

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 11.2 Addクラスの実装

In [5]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # タプルにする

In [6]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
y.data

array(5)

## ステップ１２　可変長の引数（改善編）

### 12.1 　1つ目の改善：関数を使いやすく
１つ目の改善は、関数を「使う人」にとっての改善
- ステップ１１でのコード
```
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
```
- 改善後のコード
```
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
```


In [7]:
class Function:
    def __call__(self, *inputs): # 1. アスタリスクをつける
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        # 2. リストの要素が１つの時は最初の要素を返す
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [8]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # タプルにする

In [9]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
y.data

array(5)

- memo<br>
    - <strong>可変長引数</strong><br>
        引数にアスタリスクをつけることで、リストを使わず、任意の数の引数を与えることができる。<br>
        以下具体例

In [10]:
def print_x(*x):
    print(x)

print_x(1, 2, 3)

a = 1
b = (2, 3)
print_x(a,b)

(1, 2, 3)
(1, (2, 3))


### 12.2 2つ目の改善：関数を実装しやすく
２つ目の改善は、関数を「実装する人」にとっての改善<br>
引数を直接受け取り、結果の変数も直接返すように改良
- 現状のコード
```
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)
```
- 改善後のコード
```
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
```

In [18]:
xs = [1, 2]
(*xs,)

(1, 2)

In [19]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # ①アスタリスクをつけてアンパッキング
        if not isinstance(ys, tuple): # ②タプルではない場合の追加対応
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [20]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

### 12.3 add関数の実装

In [21]:
def add(x0, x1):
    return Add()(x0, x1)

In [22]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
y.data

array(5)